# Task 1
## Baseline Document Retrieval Model
### (a)
#### Extract the text and document numbers from the corpus. You may use libraries such as BeautifulSoup or xml to parse the files. Note that the documents contain the text within 'text' tags. The LA-Times documents, whose document numbers start with LA, use additional 'P' tags.

In [1]:
#!/usr/bin/env python

import os
import re
import sys
import string
import time
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from collections import Counter, defaultdict
import xml.etree.ElementTree as ET
import regex as regE
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import math
  

#### ***** Make sure that you execute this cell only once.

- This also to append <ROOT></ROOT> tags at the beginning and end of trec_documents.xml file, so that it can be read by xml.etree.ElementTree package.

In [2]:
# Make sure that you execute this cell only once

def appendRootTags(fname):
    '''
        This function is to append <ROOT> </Root> tags at the beginning and end of trec_documents files
        so that it can be read directly by xml.etree.ElementTree package
    '''
    
    # Write <ROOT> to the beginning of the file
    with open(fname,'r') as contents:
          save = contents.read()
    with open(fname,'w') as contents:
          contents.write("<ROOT>\n")
    with open(fname,'a') as contents:
          contents.write(save)
    
    # Write </ROOT> to the beginning of the file
    f = open(fname, "a")
    
    f.write("</ROOT>")
    f.close() 

    return

appendRootTags("./trec_documents.xml")

In [3]:
def readDocuments(fname):
    '''
        This function is to read the documents and extract the document text and their id
    '''
    
    data = open(fname, "a+")
    
    data_file = ET.parse(fname)
    data_file_root = data_file.getroot()
    
    
    documents_no = []
    documents_text = []
    
    
    for node in data_file.iter('DOCNO'):
        documents_no.append(node.text.strip())
    
    for node in data_file.iter('TEXT'):
        
        if len(node.getchildren()) > 0:
            total = ""
            for node_child in node.getchildren():
                total += node_child.text
            documents_text.append(total.strip())
        else:
            documents_text.append(node.text.strip())
    
    return documents_no, documents_text   

documents_no, documents_text = readDocuments("./trec_documents.xml")
    

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


In [4]:
# An example of the first document number and first document text
print(documents_no[0])
print(documents_text[0])

FT911-5
A Malaysian English-language newspaper agreed to pay former Singapore prime
minister Lee Kuan Yew Dollars 100,000 over allegations of corruption.


### (b)
#### Pre-process the texts from the documents by applying tokenisation, lower-casing, removing punctuation tokens. You may use NLTK to tokenise the documents. However, for the rest of Section 1 you are not allowed to use any external libraries that offer functions to obtain the following weights and scores.

In [5]:
def preprocessReadDocuments(documents_text, documents_no):
    '''
        This function is to preprocess the documents texts by applying tokenization
    '''
    documents_text_tokenized = {}
    
    for i in range(len(documents_text)):
        
        documents_text_tokenized_temp = tokenize(documents_text[i])
        documents_text_tokenized[documents_no[i]] = documents_text_tokenized_temp
    
    return documents_text_tokenized

def tokenize(text):
    '''
        This function is a helper function for the preprocess function to be used to apply tokenization
    '''
    
    word_list = regE.sub(r"\p{P}","",text)
    
    word_list = word_list.lower()
    
    word_list = word_tokenize(word_list)
    
    return word_list
    

documents_text_preprocessed = preprocessReadDocuments(documents_text, documents_no)


In [6]:
from itertools import islice

def take(n, iterable):
    '''
        This function is a helper function to to be used to print certain number of elements from a dictionary
    '''
    return list(islice(iterable, n))


In [7]:
# An example of the first 10 key, value pairs of IDF data structure
# Here the key is a string representing document no.
# Here the value is a list of terms presented in the document

n_items = take(10, documents_text_preprocessed.items())
print(n_items)


[('FT911-5', ['a', 'malaysian', 'englishlanguage', 'newspaper', 'agreed', 'to', 'pay', 'former', 'singapore', 'prime', 'minister', 'lee', 'kuan', 'yew', 'dollars', '100000', 'over', 'allegations', 'of', 'corruption']), ('FT911-29', ['john', 'labatt', 'the', 'canadian', 'food', 'and', 'beverage', 'group', 'is', 'putting', 'a', 'large', 'part', 'of', 'its', 'food', 'processing', 'business', 'up', 'for', 'sale', 'in', 'keeping', 'with', 'its', 'strategy', 'of', 'concentrating', 'on', 'beer', 'entertainment', 'and', 'dairy', 'products', 'labatt', 'which', 'is', '39', 'per', 'cent', 'owned', 'by', 'the', 'brascan', 'group', 'controlled', 'by', 'peter', 'and', 'edward', 'bronfman', 'said', 'yesterday', 'it', 'will', 'consider', 'offers', 'for', 'oregonbased', 'jlfoods', 'which', 'specialises', 'in', 'frozen', 'grocery', 'products', 'in', 'canada', 'the', 'us', 'and', 'britain', 'jlfoods', 'is', 'the', 'leading', 'supplier', 'of', 'frozen', 'soup', 'in', 'the', 'us', 'and', 'also', 'a', 'larg

### (c)

#### Compute and store the inverse document frequency (idf) of the terms:
    


$$ idf(term_i) = log(\frac{N}{n(term_i}) $$

#### where N is the total number of documents and n(termi) is the number of documents that contain the termi.

In [8]:

def computeIdf(documents_text_preprocessed):
    '''
        This function is a function to calculate IDF for the terms present in documents text
    '''
    DF = {}
    IDF = {}   
    
    No_docs = len(documents_text_preprocessed)
     
    for key, value in documents_text_preprocessed.items():
        value = documents_text_preprocessed[key]
        for w in value:
            try:
                DF[w].add(key)
            except:
                DF[w] = {key}
    
    for i in DF:
        DF[i] = len(DF[i])
        
    for key, value in DF.items():
        idf = math.log(No_docs/value)
        
        IDF[key] = idf
        
    return IDF


documents_text_preprocessed_idf  = computeIdf(documents_text_preprocessed)


In [9]:
# An example of the first 10 key, value pairs of IDF data structure
# Here the key is a string representing term
# Here the value is a float representing IDF value

n_items = take(10, documents_text_preprocessed_idf.items())
print(n_items)


[('a', 0.23134949068453303), ('malaysian', 6.3686444860040545), ('englishlanguage', 6.774109594112219), ('newspaper', 3.9890983518738805), ('agreed', 3.1497686611358535), ('to', 0.24849359793474943), ('pay', 3.0199106738776402), ('former', 2.3326355007949173), ('singapore', 5.164671681678119), ('prime', 3.59189775361561)]


### (d)

#### Compute and store the term frequency (tf) of the terms per document along with the document ID:
    


$$ tf(term_i, doc) = \frac{n(term_i, doc)}{max(term_j, doc)} $$
                              
#### where n(termi,doc) is the number of times termi appears in the document and max(termj,doc) is the number of times the most frequent term of the document appears in the document.

In [10]:

def calculateTf(documents_text_preprocessed):
    '''
        This function is a function to calculate TF for the terms present in each documents text
    '''

    TF = {}
    
    for key, value in documents_text_preprocessed.items():
    
        unigrams = dict(Counter(value))
    
        v = list(unigrams.values())
        k = list(unigrams.keys())
        max_freq = max(v)

        for word, freq in unigrams.items():
            
            tf = freq / max_freq
            TF[(key, word)] = tf


    return TF

documents_text_preprocessed_tf = calculateTf(documents_text_preprocessed)


In [11]:
# An example of the first 10 key, value pairs of TF data structure
# Here the key is a tuple representing (document_no, term)
# Here the value is a float representing TF value

n_items = take(10, documents_text_preprocessed_tf.items())
print(n_items)


[(('FT911-5', 'a'), 1.0), (('FT911-5', 'malaysian'), 1.0), (('FT911-5', 'englishlanguage'), 1.0), (('FT911-5', 'newspaper'), 1.0), (('FT911-5', 'agreed'), 1.0), (('FT911-5', 'to'), 1.0), (('FT911-5', 'pay'), 1.0), (('FT911-5', 'former'), 1.0), (('FT911-5', 'singapore'), 1.0), (('FT911-5', 'prime'), 1.0)]


### (e)

#### Given a list of query terms, you can compute and store the tf-idf weights for these terms as the product of the term’s idf and the tf-value of the term in the respective document:


$$ tf-idf(term_i, doc) = tf(term_i, doc) * idf(term_i) $$

In [12]:
def calculateTfIdf(documents_text_preprocessed_idf, documents_text_preprocessed_tf):
    '''
        This function is a function to calculate TF-IDF for the terms present in each documents text
    '''
    
    
    DF = {}
    TF_IDF = {}
    
    for key, value in documents_text_preprocessed_tf.items():
        
        TF_IDF[key] = value*documents_text_preprocessed_idf[key[1]]
        

    return TF_IDF

documents_text_preprocessed_tf_idf = calculateTfIdf(documents_text_preprocessed_idf, documents_text_preprocessed_tf)


In [13]:
# An example of the first 10 key, value pairs of TF-IDF data structure
# Here the key is a tuple representing (document_no, term)
# Here the value is a float representing TF-IDF value

n_items = take(10, documents_text_preprocessed_tf_idf.items())
print(n_items)


[(('FT911-5', 'a'), 0.23134949068453303), (('FT911-5', 'malaysian'), 6.3686444860040545), (('FT911-5', 'englishlanguage'), 6.774109594112219), (('FT911-5', 'newspaper'), 3.9890983518738805), (('FT911-5', 'agreed'), 3.1497686611358535), (('FT911-5', 'to'), 0.24849359793474943), (('FT911-5', 'pay'), 3.0199106738776402), (('FT911-5', 'former'), 2.3326355007949173), (('FT911-5', 'singapore'), 5.164671681678119), (('FT911-5', 'prime'), 3.59189775361561)]


In [14]:
def calculateDocumentLength(documents_text_preprocessed, documents_text_preprocessed_idf, documents_text_preprocessed_tf):
    '''
        This function is a function to calculate TF-IDF for terms present in each documents text to be used
        to calculate the document length for the cosine similarity function later
    '''
    
    
    documents_length = {}
    curr_length = 0
    
    for key, value in documents_text_preprocessed.items():
    
        
        for term in value:
            
            tf = documents_text_preprocessed_tf[key, term]
            tf_idf = tf * documents_text_preprocessed_idf[term]
            
            curr_length = curr_length + tf_idf**2
            
        documents_length[key] = curr_length


    return documents_length

documents_length = calculateDocumentLength(documents_text_preprocessed, documents_text_preprocessed_idf, documents_text_preprocessed_tf)


In [15]:
# An example of the first 10 key, value pairs of documents-length data structure
# Here the key is a string representing the document no
# Here the value is a float representing the document length

n_items = take(10, documents_length.items())
print(n_items)


[('FT911-5', 396.07773941704494), ('FT911-29', 468.8850319979414), ('FT911-84', 569.691871751475), ('FT911-98', 694.1983159358077), ('FT911-119', 784.8882799438994), ('FT911-142', 944.7082517904695), ('FT911-149', 1174.056091448151), ('FT911-164', 1239.5244564077273), ('FT911-178', 1486.1959483319215), ('FT911-229', 1606.186193488594)]


In [16]:
def readQueries(fname):
    '''
        This function is a function to to be used to read the queries from their relevant text files
    '''
    
    data = open(fname, "r")
    
    
    queries_no = []
    queries_text = []
    
    
    for line in data:
            if '<desc>' in line:
                query = ""
                for line in data: 
                    
                    if '</top>' in line:
                        break
                    query = query + line.strip()
                    
                queries_text.append(query.strip())
    
    return queries_text

queries_text = readQueries("./test_questions.txt")

In [17]:
# An example of the first query read
print(queries_text[0])

Who is the author of the book, "The Iron Lady: A Biography of Margaret Thatcher"?


In [18]:
def preprocessReadQueries(queries_text):
    '''
        This function is to preprocess the queries texts by applying tokenization
    '''
    queries_text_tokenized = {}
    
    for i in range(len(queries_text)):
        
        queries_text_tokenized_temp = tokenize(queries_text[i])
        queries_text_tokenized[i + 1] = queries_text_tokenized_temp
    
    return queries_text_tokenized
    


In [19]:
# Preform preprocessing of the queries the same as the preprocess of the documents

queries_text_preprocessed = preprocessReadQueries(queries_text)


In [20]:
# An example of the first 10 key, value pairs of queries text preprocessed data structure
# Here the key is representing the query no
# Here the value is a of the terms in that query

n_items = take(10, queries_text_preprocessed.items())
print(n_items)


[(1, ['who', 'is', 'the', 'author', 'of', 'the', 'book', 'the', 'iron', 'lady', 'a', 'biography', 'of', 'margaret', 'thatcher']), (2, ['what', 'was', 'the', 'monetary', 'value', 'of', 'the', 'nobel', 'peace', 'prize', 'in', '1989']), (3, ['what', 'does', 'the', 'peugeot', 'company', 'manufacture']), (4, ['how', 'much', 'did', 'mercury', 'spend', 'on', 'advertising', 'in', '1993']), (5, ['what', 'is', 'the', 'name', 'of', 'the', 'managing', 'director', 'of', 'apricot', 'computer']), (6, ['why', 'did', 'david', 'koresh', 'ask', 'the', 'fbi', 'for', 'a', 'word', 'processor']), (7, ['what', 'debts', 'did', 'qintex', 'group', 'leave']), (8, ['what', 'is', 'the', 'name', 'of', 'the', 'rare', 'neurological', 'disease', 'with', 'symptoms', 'such', 'as', 'involuntary', 'movements', 'tics', 'swearing', 'and', 'incoherent', 'vocalizations', 'grunts', 'shouts', 'etc']), (9, ['how', 'far', 'is', 'yaroslavl', 'from', 'moscow']), (10, ['name', 'the', 'designer', 'of', 'the', 'shoe', 'that', 'spawned'

In [21]:
def calculateTfQueries(queries_text_preprocessed):
    '''
        This function is a function to calculate TF for the terms present in each query text
    '''
    
    TF = {}
    
    for key, value in queries_text_preprocessed.items():
    
        unigrams = dict(Counter(value))
    
        v = list(unigrams.values())
        k = list(unigrams.keys())
        max_freq = max(v)

        for word, freq in unigrams.items():
            
            tf = 0.5 + (0.5 * freq / max_freq)
            TF[(key, word)] = tf


    return TF

queries_text_preprocessed_tf = calculateTfQueries(queries_text_preprocessed)


In [22]:
# An example of the first 10 key, value pairs of TF for queries data structure
# Here the key is a tuple representing (query_no, term)
# Here the value is a float representing TF value

n_items = take(10, queries_text_preprocessed_tf.items())
print(n_items)


[((1, 'who'), 0.6666666666666666), ((1, 'is'), 0.6666666666666666), ((1, 'the'), 1.0), ((1, 'author'), 0.6666666666666666), ((1, 'of'), 0.8333333333333333), ((1, 'book'), 0.6666666666666666), ((1, 'iron'), 0.6666666666666666), ((1, 'lady'), 0.6666666666666666), ((1, 'a'), 0.6666666666666666), ((1, 'biography'), 0.6666666666666666)]


### (f)
#### Based on the tf-idf weights, you can determine the relevance of the documents for the query based on their cosine similarity with the query using dot products. To calculate the tf-idf weights for the query, you should treat it like a document.


$$ cos(q, d) = \frac{q.d}{|q||d|} $$

#### * Here in the function below, there are two approaches taken for the cosine similarity equation as follows:-
- Take the denominator of the cosine similarty equation into consideration.
    - This resulted into a very low mean average precision later, this maybe attributed to the fact that dividing by (length of the query * length of the document) results into very small numbers especially for large documents even if it contains the answer and is relevant. While small documents that have some of the same terms as the query, but don't have the answer and not relevant, it results into a much higher score although this shouldn't be correct.
    
    
- Ignore the denominator of the cosine similarity equation.
    - This resulted into a higher average precision later, this can be attributed to the fact that taking only the nominator will encourage large and relevant documents that has most of the query terms including the answer to have very high scores, since we don't normalize the score by its length. In this problem, it increased the probability of getting relevant documents and the mean average precision over the previous approach.
  
#### * This is verified later by observing the relevant documents using (patterns) data for query number 1, and found out that the most relevant document for this query is indeed larger, while there are other documents which aren't relevant but has the same query terms, and these are wrongly favoured by cosine similarity formal equation.
  
#### * The second approach is taken as it resulted into higher mean average precision, which will benefit later the more advanced model in achieving better average precision.

In [23]:
def scoreQueries(documents_text_preprocessed_tf_idf, documents_text_preprocessed_idf, documents_text_preprocessed, queries_text_preprocessed, queries_text_preprocessed_tf):
    '''
        This function is a function to calculate score for each query with each document using cosine similarity
    '''
    
    Query_TF_IDF = {}
    
    for document, doc_value in documents_text_preprocessed.items():
        
        for query_no, query_value in queries_text_preprocessed.items():
            
            cosine_nominator = 0
            query_length = 0
            
            score = 0
            
            for term in query_value:
                
                if (document, term) in documents_text_preprocessed_tf_idf:
                    cosine_nominator_curr = documents_text_preprocessed_tf_idf[(document, term)] * (queries_text_preprocessed_tf[query_no, term] * documents_text_preprocessed_idf[term])
                    cosine_nominator = cosine_nominator + cosine_nominator_curr
                    
                    query_length_curr = ((queries_text_preprocessed_tf[query_no, term] * documents_text_preprocessed_idf[term]))**2
                    query_length = query_length + query_length_curr
                    
                    
            
            if cosine_nominator != 0:
                '''
                    This equation is the formal equation of cosine similarity and given in the project pdf
                '''
                # score = cosine_nominator / (query_length * documents_length[document])
                
                '''
                    However when the denominator is ignored, it results in a much higher average precision
                '''
                
                score = cosine_nominator
                
                
            Query_TF_IDF[(document, query_no)] = score
        query_no = 0   
    return Query_TF_IDF

queries_scores = scoreQueries(documents_text_preprocessed_tf_idf, documents_text_preprocessed_idf, documents_text_preprocessed, queries_text_preprocessed, queries_text_preprocessed_tf)


In [24]:
# An example of the first 10 key, value pairs of query-scores data structure
# Here the key is a tuple representing (document_no, query_no)
# Here the value is a float representing cosine similarity score

n_items = take(10, queries_scores.items())
print(n_items)


[(('FT911-5', 1), 0.07228063506736665), (('FT911-5', 2), 0.016469509728317146), (('FT911-5', 3), 0), (('FT911-5', 4), 0), (('FT911-5', 5), 0.043918692608845726), (('FT911-5', 6), 0.053522586839992836), (('FT911-5', 7), 0), (('FT911-5', 8), 0.016469509728317146), (('FT911-5', 9), 0), (('FT911-5', 10), 0.043918692608845726)]


In [25]:
# Sort the scores of all the queries with all the documents in descending order
queries_scores_sorted = sorted(queries_scores.keys(), key=lambda x: queries_scores[x],
                     reverse=True)

In [26]:
# An example of the first 10 tuples of the query-scores-sorted data structure
# Here the first element of the tuple represents document_no
# Here the second element of the tuple represents query_no


queries_scores_sorted[:10]


[('LA061789-0021', 82),
 ('LA040790-0053', 71),
 ('FT924-14045', 16),
 ('LA030590-0027', 70),
 ('FT921-8107', 63),
 ('FT923-14280', 83),
 ('FT931-6636', 80),
 ('LA082989-0145', 49),
 ('FT922-4658', 63),
 ('FT921-7280', 97)]

### (g)
#### Sort the similarity scores and output the top 50 most relevant documents for a query along with their scores.

In [27]:
def calculateTop50Documents(queries_scores_sorted):
    '''
        This function is a function to the get highest 50 documents scores of each query
    '''

    top_50_doc_queries = {}
    for i in range(1, 101):
        top_50_doc = []
        count = 0
        for key in queries_scores_sorted:
            if count == 50:
                    break
            if key[1] == i:

                top_50_doc.append(key[0])
                count += 1
        top_50_doc_queries[i] = top_50_doc
        
    return top_50_doc_queries

top_50_doc_queries = calculateTop50Documents(queries_scores_sorted)


In [28]:
# An example of the first 10 key, value pairs of top-50-documents for each query data structure
# Here the key is the query_no
# Here the value is a list representing the numbers of the top 50 documents having
# the highest cosine similarity scores sorted in descending order

n_items = take(10, top_50_doc_queries.items())
print(n_items)


[(1, ['FT921-1445', 'FT932-3036', 'FT923-1864', 'FT931-16637', 'LA100990-0049', 'FT941-9689', 'FT931-4680', 'FT943-6188', 'LA103090-0044', 'FT924-6263', 'FT911-5242', 'FT924-16147', 'LA100490-0041', 'FT934-14583', 'FT931-14768', 'LA040189-0021', 'LA082889-0101', 'LA062689-0064', 'FT943-7419', 'LA082789-0115', 'FT931-9284', 'LA081290-0080', 'LA111890-0122', 'FT924-3340', 'FT921-13302', 'FT923-1084', 'LA092390-0186', 'LA101089-0018', 'LA082289-0039', 'FT911-300', 'FT942-3972', 'FT932-1679', 'FT934-15521', 'FT941-10164', 'LA081290-0079', 'FT941-3410', 'LA030589-0099', 'LA040790-0152', 'FT923-9022', 'LA061189-0196', 'FT911-784', 'LA011989-0197', 'FT944-9877', 'LA051590-0157', 'FT941-14165', 'FT934-11714', 'FT944-13298', 'FT924-5587', 'FT934-11397', 'FT931-6126']), (2, ['FT933-7324', 'FT943-317', 'LA102490-0031', 'FT911-2323', 'FT931-10284', 'LA050890-0174', 'FT934-17465', 'FT943-12587', 'LA050589-0109', 'FT924-14045', 'FT923-1291', 'FT921-10516', 'LA012089-0088', 'LA051090-0085', 'FT941-15

In [29]:
# Based on the previous result, here is an exmaple of the top document scored for query number 1, 
# it's document number 'FT921-1445'

print("***** Query No. 1 *****\n")
print(queries_text_preprocessed[1])
print("\n")
print("***** Highest scored document *****\n")
print(documents_text_preprocessed['FT921-1445'])

***** Query No. 1 *****

['who', 'is', 'the', 'author', 'of', 'the', 'book', 'the', 'iron', 'lady', 'a', 'biography', 'of', 'margaret', 'thatcher']


***** Highest scored document *****

['the', 'economist', 'known', 'as', 'the', 'father', 'of', 'monetarism', 'and', 'a', 'guru', 'of', 'former', 'prime', 'minister', 'margaret', 'thatcher', 'professor', 'friedrick', 'von', 'hayek', 'died', 'at', 'his', 'home', 'in', 'freiburg', 'in', 'breisgau', 'germany', 'aged', '92']


### We observe that the highest score document has indeed some of the same terms as the query, but it doesn't have the required answer. The answer should be "Hugo Young"
#### This will be improved later when using BM25 score for the top retrieved 1000 documents

In [30]:
def readPatterns(fname):
    '''
        This function is a function to read the patterns text file and for each query append the relevant answer patterns
    '''
    
    data = open(fname, "r")
    
    
    patterns = {}
    
    
    for line in data:
            
        line = line.strip()
        line = line.split()
        
        if line[0] in patterns:
            patterns[line[0]].append(line[1])
        else:
            patterns[line[0]] = [line[1]]
        
    
    return patterns

patterns = readPatterns("./patterns.txt")

In [31]:
# An example of the first 10 key, value pairs of patterns data structure
# Here the key is the query_no
# Here the value is a list representing the patterns associated with
# the corresponding query

n_items = take(10, patterns.items())
print(n_items)

[('1', ['Young']), ('2', ['\\$469,000']), ('3', ['405', 'automobiles?', 'diesel\\s+motors?', '309s?', '106s?', '504s?', '505s?', '205s?', '306s?', 'vehicles?', 'cars?', 'Peugeots', 'plastic\\s+components']), ('4', ['Pounds\\s+12\\s*(?:m|(?:million))']), ('5', ['Horne']), ('6', ['To\\s+record\\s+his\\s+revelations', 'finish\\w*\\s+writing.*?revelations']), ('7', ['1\\.4\\s*(?:(?:bn)|(?:billion))', '1\\.6\\s*(?:(?:bn)|(?:billion))']), ('8', ["Tourette\\s*'\\s*s"]), ('9', ['150']), ('10', ['Pfister'])]


In [32]:
def getRelevantDocs(patterns, documents_no, documents_text, number_query):
    '''
        This function is a function to get the relevant documents for each query based on the associated patterns
    '''
    
    relevant_docs = {}
    
    for i in range(1, number_query + 1):
        
        relevant_docs[i] = []
        for k in range(len(patterns[str(i)])):
            regex = re.compile(patterns[str(i)][k])
            for j in range(len(documents_text)):
                if regex.search(documents_text[j]) != None:
                    
                    relevant_docs[i].append(documents_no[j])
                    
                    
    return relevant_docs

relevant_docs = getRelevantDocs(patterns, documents_no, documents_text, len(queries_text_preprocessed))


In [33]:
# An example of the first 10 key, value pairs of relevant-docs data structure
# Here the key is the query_no
# Here the value is a list representing the documents numbers relevant to
# the corresponding query_no

n_items = take(10, relevant_docs.items())
print(n_items)

[(1, ['FT911-1135', 'FT921-6562', 'FT921-502', 'FT923-9014', 'FT924-11506', 'FT924-14313', 'FT931-3210', 'FT931-3314', 'FT931-6403', 'FT931-6632', 'FT931-13345', 'FT931-2961', 'FT931-3136', 'FT931-3146', 'FT932-4042', 'FT932-5580', 'FT932-484', 'FT932-11063', 'FT932-15484', 'FT932-1443', 'FT933-11245', 'FT933-14023', 'FT933-15156', 'FT934-6213', 'FT934-357', 'FT934-368', 'FT934-6676', 'FT934-6917', 'FT934-9371', 'FT934-9374', 'FT934-9628', 'FT934-10110', 'FT934-12477', 'FT934-14924', 'FT934-16697', 'FT941-962', 'FT941-969', 'FT941-11086', 'FT941-13098', 'FT941-1941', 'FT941-2349', 'FT942-237', 'FT942-4862', 'FT942-5087', 'FT942-5732', 'FT942-9366', 'FT942-12067', 'FT942-15505', 'FT942-1537', 'FT943-4159', 'FT943-12303', 'FT943-12564', 'FT943-15240', 'FT944-7556', 'FT944-10751', 'FT944-11169', 'FT944-1726', 'LA010289-0005', 'LA010590-0016', 'LA020390-0006', 'LA020589-0052', 'LA021189-0097', 'LA021290-0066', 'LA021290-0141', 'LA021589-0036', 'LA021590-0082', 'LA030589-0026', 'LA031390-00

In [34]:
# Here we check if highest scored document 'FT921-1445' for query number 1 exists in the relevant documents or not

print("***** Query No. 1 *****\n")
print(queries_text_preprocessed[1])
print("\n")
print("***** Highest scored document *****\n")
for value in relevant_docs[1]:
    
    if value == top_50_doc_queries[1][0]:
        print("Document number: ", value)
        

***** Query No. 1 *****

['who', 'is', 'the', 'author', 'of', 'the', 'book', 'the', 'iron', 'lady', 'a', 'biography', 'of', 'margaret', 'thatcher']


***** Highest scored document *****



### We observe that the highest score document for query number 1 doesn't exist in the relevant documents
#### This will change and improve when using bm25 scores

In [35]:
# This loop is just to get the number of queries which doesn't have relevant documents

count = 0
for key, value in relevant_docs.items():
    if len(value) == 0:
        count += 1

print(count)

6


### (h)
#### Write a function to evaluate the performance of your document using the evaluation metric precision at r with r = 50:


$$ precision = \frac{n(relevant and retrieved documents)}{n(retrieved documents)} $$


#### Precision at r computes the percentage of relevant documents from the top r retrieved documents. You should ignore the documents ranked lower than r

In [36]:
def calculatePrecisionAtR(top_50_doc_queries, relevant_docs, query_no, rank):
    '''
        This function is a function to calculate the average precision of all queries with max rank 50
    '''
        
    
    count = 0
    precision = 0
    
    if top_50_doc_queries[query_no][rank - 1] in relevant_docs[query_no]:
        for index, document in enumerate(top_50_doc_queries[query_no]):

            if index == rank:
                break
            if document in relevant_docs[query_no]:
                count = count + 1
            
    
    precision = (count / rank)
    
    return precision



In [37]:

# An example precision at r = 50 calculated for the third query
# It has precision of 0.62 at rank 50

precision_at_r_first_query = calculatePrecisionAtR(top_50_doc_queries, relevant_docs, 3, 50)
print(precision_at_r_first_query)

0.62


### (i)

#### Use the precision at r function to evaluate your performance as mean of precisions over the queries given in test_questions.txt. First, you must extract the queries from the descriptions. A document is relevant, if it contains the answer. You can check this by using the regex patterns for the answers to the corresponding queries in patterns.txt.

In [38]:
def getNumberRelevantDocuments(top_50_doc_queries, relevant_docs, query_no):
    '''
        This function is a helper function for the following function to calculate the number of relevant documents
        for a given query
    '''
  
    count = 0
     
    for index, document in enumerate(top_50_doc_queries[query_no]):

        if document in relevant_docs[query_no]:
            count = count + 1
    
    return count



In [39]:
def calculateAveragePrecision(top_50_doc_queries, relevant_docs, R):
    '''
        This function is a function to calculate the average precision of all queries with max rank 50
    '''
    
    total_precision = 0
    
    for i in range(1, len(top_50_doc_queries) + 1):
        
        precision_at_query = 0
        no_relevant_docs = 0
        
        for r in range(1, R + 1):
            
            precision = calculatePrecisionAtR(top_50_doc_queries, relevant_docs, i, r)
            
            precision_at_query += precision
        
        no_relevant_docs = getNumberRelevantDocuments(top_50_doc_queries, relevant_docs, i)
        
        if no_relevant_docs != 0:
            
            total_precision += (precision_at_query / no_relevant_docs)
        
    return total_precision / 100



In [40]:
# Calculate the average precision for all the queries

mean_average_precision = calculateAveragePrecision(top_50_doc_queries, relevant_docs, 50)
print(mean_average_precision)

0.164375270553892


# Task 2
## Advanced Document Retriever with Re-Ranking
### (a)
#### Use the baseline model from Section 1 and return the top 1000 documents given a query.

In [41]:
def calculateTop1000Documents(queries_scores_sorted):
    '''
        This function is a function to get the top 1000 documents numbers for each query
    '''

    top_1000_doc_queries = {}
    for i in range(1, 101):
        top_1000_doc = []
        count = 0
        for key in queries_scores_sorted:
            if count == 1000:
                    break
            if key[1] == i:

                top_1000_doc.append(key[0])
                count += 1
        top_1000_doc_queries[i] = top_1000_doc
        
    return top_1000_doc_queries

top_1000_doc_queries = calculateTop1000Documents(queries_scores_sorted)


In [42]:
# An example of the first 10 key, value pairs of top_1000_doc_queries data structure
# Here the key is the query_no
# Here the value is a list representing the documents numbers for the top 1000
# documents with the highest cosine similarity scores sorted descendingly

n_items = take(10, top_1000_doc_queries.items())
print(n_items)

[(1, ['FT921-1445', 'FT932-3036', 'FT923-1864', 'FT931-16637', 'LA100990-0049', 'FT941-9689', 'FT931-4680', 'FT943-6188', 'LA103090-0044', 'FT924-6263', 'FT911-5242', 'FT924-16147', 'LA100490-0041', 'FT934-14583', 'FT931-14768', 'LA040189-0021', 'LA082889-0101', 'LA062689-0064', 'FT943-7419', 'LA082789-0115', 'FT931-9284', 'LA081290-0080', 'LA111890-0122', 'FT924-3340', 'FT921-13302', 'FT923-1084', 'LA092390-0186', 'LA101089-0018', 'LA082289-0039', 'FT911-300', 'FT942-3972', 'FT932-1679', 'FT934-15521', 'FT941-10164', 'LA081290-0079', 'FT941-3410', 'LA030589-0099', 'LA040790-0152', 'FT923-9022', 'LA061189-0196', 'FT911-784', 'LA011989-0197', 'FT944-9877', 'LA051590-0157', 'FT941-14165', 'FT934-11714', 'FT944-13298', 'FT924-5587', 'FT934-11397', 'FT931-6126', 'FT942-15904', 'LA120690-0199', 'LA120690-0197', 'FT931-10865', 'LA010990-0071', 'FT944-13412', 'FT934-13802', 'FT931-14845', 'FT944-13515', 'LA041590-0008', 'FT944-7647', 'LA040989-0153', 'FT942-6814', 'FT933-732', 'FT943-4661', '

### (b)
#### Re-rank the top 1000 documents with a more advanced approach. As inspiration, you can look at more advanced scoring functions such as BM25, SVMs for ranking or experiment with neural re-ranking methods. Your re-ranker should return the top 50 documents based on the top 1000 documents returned in (a).

In [43]:
def computeIdfForBM25(documents_text_preprocessed):
    '''
        This function is a function to calculate IDF of bm25 for the terms present in documents text
    '''
    DF = {}
    IDF = {}

    
    No_docs = len(documents_text_preprocessed)
    
    
    
    for key, value in documents_text_preprocessed.items():
        value = documents_text_preprocessed[key]
        for w in value:
            try:
                DF[w].add(key)
            except:
                DF[w] = {key}
    
    for key in DF:
        DF[key] = len(DF[key])
        
    for key, value in DF.items():
        idf = math.log((No_docs - value + 0.5) / (value + 0.5) +1)
        
        IDF[key] = idf
        

    return IDF


documents_text_preprocessed_idf_bm25  = computeIdfForBM25(documents_text_preprocessed)


In [44]:
# An example of the first 10 key, value pairs of IDF of bm25 data structure
# Here the key is a string representing term
# Here the value is a float representing IDF value

n_items = take(10, documents_text_preprocessed_idf_bm25.items())
print(n_items)


[('a', 0.23139176017070548), ('malaysian', 6.335968955426459), ('englishlanguage', 6.725433722188183), ('newspaper', 3.986130977581868), ('agreed', 3.148550508147471), ('to', 0.24853462205540983), ('pay', 3.018854690974845), ('former', 2.3321610382396734), ('singapore', 5.154835643070346), ('prime', 3.5899395062590327)]


In [45]:
def calculateTfForBM25(documents_text_preprocessed):
    '''
        This function is a function to calculate TF of bm25 for the terms present in each documents text
    '''
    
    TF = {}
    
    for key, value in documents_text_preprocessed.items():
    
        unigrams = dict(Counter(value))
    
        v = list(unigrams.values())
        k = list(unigrams.keys())
        max_freq = float(max(v))



        for word, freq in unigrams.items():
            
            tf = freq
            TF[(key, word)] = tf


    return TF

documents_text_preprocessed_tf_bm25 = calculateTfForBM25(documents_text_preprocessed)


In [46]:
# An example of the first 10 key, value pairs of TF for bm25 data structure
# Here the key is a tuple representing (document_no, term)
# Here the value is a float representing TF value

n_items = take(10, documents_text_preprocessed_tf_bm25.items())
print(n_items)


[(('FT911-5', 'a'), 1), (('FT911-5', 'malaysian'), 1), (('FT911-5', 'englishlanguage'), 1), (('FT911-5', 'newspaper'), 1), (('FT911-5', 'agreed'), 1), (('FT911-5', 'to'), 1), (('FT911-5', 'pay'), 1), (('FT911-5', 'former'), 1), (('FT911-5', 'singapore'), 1), (('FT911-5', 'prime'), 1)]


In [47]:
def scoreBM25Queries(top_1000_doc_queries, documents_text_preprocessed_tf_bm25, documents_text_preprocessed_idf_bm25, documents_text_preprocessed, queries_text_preprocessed):
    '''
        This function is a function to calculate score for each query with 
        each document using bm25 metric
    '''
    Query_BM25_scores = {}
    
    # Hyperparamters for BM25 equation
    k = 1.2
    b = 0.75
    
    for query_no, query_value in queries_text_preprocessed.items():
        
        top_1000_document = top_1000_doc_queries[query_no]
        
        documents_text_preprocessed_filtered = {k: documents_text_preprocessed[k] for k in top_1000_document if k in top_1000_document}
        
        documents_text_preprocessed_idf_bm25_filtered  = computeIdfForBM25(documents_text_preprocessed_filtered)
        
        documents_text_preprocessed_tf_bm25_filtered = calculateTfForBM25(documents_text_preprocessed_filtered)
        
        avg_doc_len = 0
        
        for document in top_1000_document:
            avg_doc_len += len(documents_text_preprocessed[document])
            
        avg_doc_len = avg_doc_len / len(top_1000_document)
        
        for document in top_1000_document:
            score = 0
            for term in query_value:
                
                if (document, term) in documents_text_preprocessed_tf_bm25_filtered:
                    
                    tf = documents_text_preprocessed_tf_bm25_filtered[(document, term)]
                    idf = documents_text_preprocessed_idf_bm25_filtered[term]
                    
                    doc_len = len(documents_text_preprocessed[document]) / avg_doc_len
                    
                    bm25 = idf * ((tf * (k + 1)) / (tf + k * (1 - b + b * doc_len)))
                    
                    score = score + bm25
            Query_BM25_scores[(document, query_no)] = score
        
    return Query_BM25_scores


queries_scores_bm25 = scoreBM25Queries(top_1000_doc_queries, documents_text_preprocessed_tf_bm25, documents_text_preprocessed_idf_bm25, documents_text_preprocessed, queries_text_preprocessed)


In [48]:
# An example of the first 10 key, value pairs of query-scores bm25 data structure
# Here the key is a tuple representing (document_no, query_no)
# Here the value is a float representing bm25 score


n_items = take(10, queries_scores_bm25.items())
print(n_items)


[(('FT921-1445', 1), 11.771775526387232), (('FT932-3036', 1), 12.166064723347805), (('FT923-1864', 1), 10.806398428133384), (('FT931-16637', 1), 5.838112692473927), (('LA100990-0049', 1), 5.683288428454883), (('FT941-9689', 1), 5.42562791615034), (('FT931-4680', 1), 5.258486584290051), (('FT943-6188', 1), 5.001388643599586), (('LA103090-0044', 1), 5.001388643599586), (('FT924-6263', 1), 5.852371207109102)]


In [49]:
# Sort the scores of all the queries with all the documents in descending order

queries_scores_bm25_sorted = sorted(queries_scores_bm25.keys(), key=lambda x: queries_scores_bm25[x],
                     reverse=True)

In [50]:
# An example of the first 10 tuples of the query-scores-sorted data structure
# Here the first element of the tuple represents document_no
# Here the second element of the tuple represents query_no

queries_scores_bm25_sorted[:10]


[('LA062890-0007', 26),
 ('LA101289-0111', 62),
 ('LA062589-0077', 44),
 ('LA121690-0224', 8),
 ('FT924-14045', 16),
 ('LA031589-0004', 97),
 ('FT932-13943', 19),
 ('FT921-10204', 26),
 ('FT923-437', 80),
 ('LA091390-0084', 62)]

In [51]:
# An example of the first 10 key, value pairs of top-50-documents for each query data structure
# Here the key is the query_no
# Here the value is a list representing the numbers of the top 50 documents having
# the highest bm25 scores sorted in descending order

top_50_doc_queries_bm25 = calculateTop50Documents(queries_scores_bm25_sorted)

n_items = take(10, top_50_doc_queries_bm25.items())
print(n_items)


[(1, ['LA111289-0002', 'LA030589-0099', 'FT934-13802', 'LA062689-0064', 'LA040790-0152', 'FT911-5242', 'LA091990-0020', 'FT932-3036', 'LA051590-0157', 'FT921-1445', 'LA090290-0118', 'LA120690-0199', 'LA112890-0035', 'LA110590-0142', 'FT923-1864', 'LA011689-0111', 'LA082789-0115', 'FT941-14165', 'FT944-13515', 'LA043089-0012', 'LA102189-0045', 'LA110890-0129', 'LA022190-0072', 'LA082889-0101', 'LA021690-0175', 'FT943-101', 'FT911-270', 'LA072090-0135', 'LA040189-0021', 'FT921-13302', 'LA111590-0256', 'LA091489-0084', 'LA101589-0001', 'FT931-10865', 'LA072489-0070', 'FT931-15780', 'LA061189-0196', 'LA121589-0192', 'LA100490-0041', 'LA041390-0192', 'LA010990-0071', 'LA120390-0062', 'LA011989-0197', 'FT944-17588', 'LA031290-0133', 'LA110289-0038', 'LA050390-0229', 'LA011289-0153', 'LA042389-0115', 'LA072090-0045']), (2, ['LA072290-0038', 'LA010590-0170', 'LA101889-0032', 'FT943-317', 'LA100589-0216', 'LA102490-0031', 'FT934-14533', 'FT934-15808', 'FT944-16821', 'LA112890-0145', 'LA092790-0

In [52]:
# Based on the previous result, here is an exmaple of the top document scored for query number 1, 
# it's document number 'LA111289-0002'

print("***** Query No. 1 *****\n")
print(queries_text_preprocessed[1])
print("\n")
print("***** Highest scored document *****\n")
print(documents_text_preprocessed['LA111289-0002'])

***** Query No. 1 *****

['who', 'is', 'the', 'author', 'of', 'the', 'book', 'the', 'iron', 'lady', 'a', 'biography', 'of', 'margaret', 'thatcher']


***** Highest scored document *****

['there', 'are', 'two', 'aspects', 'of', 'the', 'margaret', 'thatcher', 'phenomenon', 'neither', 'of', 'which', 'is', 'addressed', 'in', 'this', 'splendid', 'british', 'biography', 'that', 'every', 'thinking', 'american', 'needs', 'to', 'keep', 'in', 'mind', 'when', 'assessing', 'the', 'achievements', 'of', 'this', 'remarkable', 'woman', 'first', 'is', 'her', 'contribution', 'to', 'global', 'diplomacy', 'a', 'quarter', 'of', 'a', 'century', 'after', 'dean', 'achesons', 'remark', 'that', 'great', 'britain', 'has', 'lost', 'an', 'empire', 'and', 'has', 'not', 'yet', 'found', 'a', 'role', 'as', 'trumans', 'secretary', 'of', 'state', 'he', 'would', 'have', 'known', 'thatcher', 'has', 'created', 'a', 'noble', 'task', 'for', 'her', 'country', 'keeping', 'american', 'foreign', 'policy', 'on', 'course', 'when'

### We observe that the highest score document has indeed some of the same terms as the query, but also has the answer to the required query
#### The answer should be "Hugo Young"

In [53]:
# Here we check if this document 'LA111289-0002' exists in the relevant documents or not

print("***** Query No. 1 *****\n")
print(queries_text_preprocessed[1])
print("\n")
print("***** Highest scored document *****\n")
for value in relevant_docs[1]:
    
    if value == top_50_doc_queries_bm25[1][0]:
        print("Document number: ", value)
        

***** Query No. 1 *****

['who', 'is', 'the', 'author', 'of', 'the', 'book', 'the', 'iron', 'lady', 'a', 'biography', 'of', 'margaret', 'thatcher']


***** Highest scored document *****

Document number:  LA111289-0002


### We observe that the highest score documeny for query number 1 is indeed exists in the relevant documents


In [54]:
# Print length of the document that scored the highest for query number 1 using Baseline Model 'FT921-1445'
# which isn't relevant, and the length of the document that scored the highest for the same query using the Advanced 
# Model 'LA111289-0002' which is a relevant document

print(documents_length['FT921-1445'])
print(documents_length['LA111289-0002'])

54988.498987184
1282454.276859744


#### This maybe an indication for the point mentioned before about the cosine similarity equation. Relevant documents that have the answer are longer than non-relevant documents that don't have the answer but still score high since they have overlapping query terms

In [55]:
# Calculate the average precision for all the queries

average_precision_bm25 = calculateAveragePrecision(top_50_doc_queries_bm25, relevant_docs, 50)
print(average_precision_bm25)

0.46216441301354044


# Task 3
## Sentence Ranker
### (a)
#### Split the top 50 documents into sentences. You may use an existing function such as sent_tokenize from NLTK.

In [56]:
def readDocumentsWithoutTokenization(documents_text, documents_no):
    '''
        This function is to read the documents without applying tokenization
        It is a helper function for the getSentencesFromTop50Documents()
    '''
    documents_text_plain = {}
    
    for i in range(len(documents_text)):
        
        documents_text_temp = documents_text[i]
        documents_text_plain[documents_no[i]] = documents_text_temp
    
    return documents_text_plain

documents_text_not_tokenized = readDocumentsWithoutTokenization(documents_text, documents_no)

In [57]:
# An example of the first 10 key, value pairs of documents without tokenization data structure
# Here the key is a string representing document no.
# Here the value is a string representing document text

n_items = take(10, documents_text_not_tokenized.items())
print(n_items)

[('FT911-5', 'A Malaysian English-language newspaper agreed to pay former Singapore prime\nminister Lee Kuan Yew Dollars 100,000 over allegations of corruption.'), ('FT911-29', "JOHN LABATT, the Canadian food and beverage group, is putting a large part\nof its food processing business up for sale in keeping with its strategy of\nconcentrating on beer, entertainment and dairy products.\nLabatt, which is 39 per cent owned by the Brascan group controlled by Peter\nand Edward Bronfman, said yesterday it will consider offers for Oregon-based\nJLFoods, which specialises in frozen grocery products in Canada, the US and\nBritain. JLFoods is the leading supplier of frozen soup in the US, and also\na large supplier of breaded and battered products.\nThe company has annual sales of USDollars 475m and employs 3,500 people at\n13 manufacturing and processing plants, most of them in the US.\nAnalysts said the sale could fetch about USDollars 300m. The company, which\nhad sales of CDollars 5.3bn (USD

In [58]:
def preprocessReadSentences(documents_text):
    '''
        This function is to preprocess the documents by converting it to sentences then applying tokenization.
        It is a helper function for the getSentencesFromTop50Documents()
    '''
    
    documents_text_sentences = []
    
    for i in range(len(documents_text)):
        
        documents_text_sentences_temp = nltk.sent_tokenize(documents_text[i])
        documents_text_sentences.append(documents_text_sentences_temp)
    
    
    documents_text_sentences = [val for sublist in documents_text_sentences for val in sublist]
    
    documents_text_sentences_tokenized = []
    
    for i in range(len(documents_text_sentences)):
        
        documents_text_sentences_tokenized_temp = tokenize(documents_text_sentences[i])
        documents_text_sentences_tokenized.append(documents_text_sentences_tokenized_temp)
    
        
    
    return documents_text_sentences_tokenized


sentences_text_preprocessed = preprocessReadSentences(documents_text)


In [59]:
# An example of the first 10 sentences extracted from all the documents

sentences_text_preprocessed[:10]

[['a',
  'malaysian',
  'englishlanguage',
  'newspaper',
  'agreed',
  'to',
  'pay',
  'former',
  'singapore',
  'prime',
  'minister',
  'lee',
  'kuan',
  'yew',
  'dollars',
  '100000',
  'over',
  'allegations',
  'of',
  'corruption'],
 ['john',
  'labatt',
  'the',
  'canadian',
  'food',
  'and',
  'beverage',
  'group',
  'is',
  'putting',
  'a',
  'large',
  'part',
  'of',
  'its',
  'food',
  'processing',
  'business',
  'up',
  'for',
  'sale',
  'in',
  'keeping',
  'with',
  'its',
  'strategy',
  'of',
  'concentrating',
  'on',
  'beer',
  'entertainment',
  'and',
  'dairy',
  'products'],
 ['labatt',
  'which',
  'is',
  '39',
  'per',
  'cent',
  'owned',
  'by',
  'the',
  'brascan',
  'group',
  'controlled',
  'by',
  'peter',
  'and',
  'edward',
  'bronfman',
  'said',
  'yesterday',
  'it',
  'will',
  'consider',
  'offers',
  'for',
  'oregonbased',
  'jlfoods',
  'which',
  'specialises',
  'in',
  'frozen',
  'grocery',
  'products',
  'in',
  'canada'

In [60]:
def getSentencesFromTop50Documents(top_50_doc_queries_bm25, documents_text_not_tokenized):
    '''
        This function is to get the tokenized sentences of top 50 documents for each query
    '''
    
    
    top_50_doc_sentences_queries_bm25 = {}
    
    for key, value in top_50_doc_queries_bm25.items():
        documents_top_50 = []
        for document_id in value:
            documents_top_50.append(documents_text_not_tokenized[document_id])
        documents_top_50_sentences = preprocessReadSentences(documents_top_50)
        top_50_doc_sentences_queries_bm25[key] = documents_top_50_sentences
        
    return top_50_doc_sentences_queries_bm25

top_50_doc_sentences_queries_bm25 = getSentencesFromTop50Documents(top_50_doc_queries_bm25, documents_text_not_tokenized)


In [61]:
# An example of the first key, value pairs of tokenized sentences of top 50 documents for each query data structure
# Here the key is a string representing query no.
# Here the value is a list of lists of strings representing sentences terms

n_items = take(1, top_50_doc_sentences_queries_bm25.items())
print(n_items)

[(1, [['there', 'are', 'two', 'aspects', 'of', 'the', 'margaret', 'thatcher', 'phenomenon', 'neither', 'of', 'which', 'is', 'addressed', 'in', 'this', 'splendid', 'british', 'biography', 'that', 'every', 'thinking', 'american', 'needs', 'to', 'keep', 'in', 'mind', 'when', 'assessing', 'the', 'achievements', 'of', 'this', 'remarkable', 'woman'], ['first', 'is', 'her', 'contribution', 'to', 'global', 'diplomacy'], ['a', 'quarter', 'of', 'a', 'century', 'after', 'dean', 'achesons', 'remark', 'that', 'great', 'britain', 'has', 'lost', 'an', 'empire', 'and', 'has', 'not', 'yet', 'found', 'a', 'role', 'as', 'trumans', 'secretary', 'of', 'state', 'he', 'would', 'have', 'known', 'thatcher', 'has', 'created', 'a', 'noble', 'task', 'for', 'her', 'country', 'keeping', 'american', 'foreign', 'policy', 'on', 'course'], ['when', 'the', 'bush', 'administration', 'was', 'struggling', 'to', 'acquire', 'its', 'sea', 'legs', 'earlier', 'this', 'year', 'it', 'was', 'thatcher', 'who', 'traveled', 'to', 'so

### (b)
#### Using the same approach you developed in Section 2, you can treat the sentences like documents to rank them and return the top 50 sentences.

In [62]:
def computeIdfForBM25Sentences(top_50_document_sentences_query):
    '''
        This function is to calculate IDF of terms in all the sentences of the top 50 documents for a given query
    '''
    IDF = {}
    
    No_sentences = len(top_50_document_sentences_query)
    
    top_50_document_sentences_with_keys = {}
    
    SF = {}
    
    key = 0
    
    for sentence in top_50_document_sentences_query:
        top_50_document_sentences_with_keys[key] = sentence
        key = key + 1
    
   
    for key, sentence in top_50_document_sentences_with_keys.items():
        
        for w in sentence:
            try:
                SF[w].add(key)
            except:
                SF[w] = {key}
    
    for key in SF:
        SF[key] = len(SF[key])
        
    for key, value in SF.items():
        idf = math.log((No_sentences - value + 0.5) / (value + 0.5) +1)
        
        IDF[key] = idf
        

    return IDF



sentences_text_preprocessed_idf_bm25  = computeIdfForBM25Sentences(top_50_doc_sentences_queries_bm25[1])



In [63]:
# An example of the first 10 key, value pairs of IDF of sentences terms of given query data structure
# Here the key is a string representing term
# Here the value is a float representing IDF value


n_items = take(10, sentences_text_preprocessed_idf_bm25.items())
print(n_items)


[('there', 3.633751063942443), ('are', 2.7066103160442223), ('two', 3.350624808026523), ('aspects', 6.542471960506805), ('of', 0.8420283871161179), ('the', 0.4384230526516766), ('margaret', 3.790936647464856), ('thatcher', 3.130224742658064), ('phenomenon', 6.542471960506805), ('neither', 5.443859671838695)]


In [64]:
def computeTfForBM25Sentences(top_50_document_sentences):
    '''
        This function is to calculate TF of terms in all the sentences of the top 50 documents for a given query
    '''
    
    TF = {}
    
    key = 0
    
    for value in top_50_document_sentences:
    
        unigrams = dict(Counter(value))
        if value:
            v = list(unigrams.values())
            k = list(unigrams.keys())
            max_freq = max(v)



            for word, freq in unigrams.items():

                tf = freq
                TF[(key, word)] = tf

        key = key + 1
    return TF


sentences_text_preprocessed_tf_bm25  = computeTfForBM25Sentences(top_50_doc_sentences_queries_bm25[1])


In [65]:
# An example of the first 10 key, value pairs of TF of sentences terms of given query data structure
# Here the key is a tuple representing sentence no. and term
# Here the value is a float representing TF value


n_items = take(10, sentences_text_preprocessed_tf_bm25.items())
print(n_items)


[((0, 'there'), 1), ((0, 'are'), 1), ((0, 'two'), 1), ((0, 'aspects'), 1), ((0, 'of'), 3), ((0, 'the'), 2), ((0, 'margaret'), 1), ((0, 'thatcher'), 1), ((0, 'phenomenon'), 1), ((0, 'neither'), 1)]


In [66]:
def scoreBM25QueriesSentences(top_50_doc_sentences_queries_bm25):
    '''
        This function is a function to calculate score for each query with 
        each sentence using bm25 metric
    '''
    Query_BM25_scores = {}
    
    # Hyperparamters for BM25 equation
    k = 2
    b = 0.75
    
    for query_no, query_value in queries_text_preprocessed.items():
        
        top_50_document_sentences = top_50_doc_sentences_queries_bm25[query_no]
        
        
        sentences_text_preprocessed_idf_filtered  = computeIdfForBM25Sentences(top_50_document_sentences)
        
        sentences_text_preprocessed_tf_filtered = computeTfForBM25Sentences(top_50_document_sentences)
        
        avg_sentences_len = 0
        
        for sentence in top_50_document_sentences:
            avg_sentences_len += len(sentence)
            
        avg_sentences_len = avg_sentences_len / len(top_50_document_sentences)
        
        for sentence in range(len(top_50_document_sentences)):
            score = 0
            for term in query_value:
                
                if (sentence, term) in sentences_text_preprocessed_tf_filtered:
                    
                    tf = sentences_text_preprocessed_tf_filtered[(sentence, term)]
                    idf = sentences_text_preprocessed_idf_filtered[term]
                    
                    sen_len = len(top_50_document_sentences[sentence]) / avg_sentences_len
                    
                    bm25 = idf * ((tf * (k + 1)) / (tf + k * (1 - b + b * sen_len)))
                    
                    score = score + bm25
            Query_BM25_scores[(sentence, query_no)] = score
        
    return Query_BM25_scores

queries_scores_bm25_sentences = scoreBM25QueriesSentences(top_50_doc_sentences_queries_bm25)


In [67]:
# An example of the first 10 key, value pairs of query-scores bm25 for sentences data structure
# Here the key is a tuple representing (sentence_no, query_no)
# Here the value is a float representing bm25 score


n_items = take(20, queries_scores_bm25_sentences.items())
print(n_items)

[((0, 1), 12.327273806176139), ((1, 1), 2.2560630535335515), ((2, 1), 4.977877726036736), ((3, 1), 5.6482808765154715), ((4, 1), 5.227450473993749), ((5, 1), 7.233009042328922), ((6, 1), 0), ((7, 1), 0), ((8, 1), 4.115201392464733), ((9, 1), 3.702230253366073), ((10, 1), 2.331131185272303), ((11, 1), 8.997752265525017), ((12, 1), 7.349263123826091), ((13, 1), 7.709106261389117), ((14, 1), 3.7712792326530833), ((15, 1), 8.119054812880073), ((16, 1), 8.22867268880337), ((17, 1), 10.618390999128877), ((18, 1), 9.863226185369387), ((19, 1), 1.7632820633988973)]


In [68]:
# Sort the scores of all the queries with all the sentences in descending order


queries_scores_bm25_sorted_sentences = sorted(queries_scores_bm25_sentences.keys(), key=lambda x: queries_scores_bm25_sentences[x],
                     reverse=True)

In [69]:
# An example of the first 10 tuples of the query-scores-sorted for sentences data structure
# Here the first element of the tuple represents sentence_no
# Here the second element of the tuple represents query_no


queries_scores_bm25_sorted_sentences[:10]

[(16, 8),
 (28, 44),
 (17, 99),
 (7, 80),
 (28, 13),
 (20, 10),
 (12, 62),
 (0, 71),
 (6, 60),
 (11, 58)]

In [70]:
def calculateTop50Sentences(queries_scores_bm25_sorted_sentences):
    '''
        This function is a function to the get highest 50 sentences scores of each query
    '''


    top_50_sen_queries = {}
    for i in range(1, 101):
        top_50_sen = []
        count = 0
        for key in queries_scores_bm25_sorted_sentences:
            if count == 50:
                    break
            if key[1] == i:

                top_50_sen.append(key[0])
                count += 1
        top_50_sen_queries[i] = top_50_sen
        
    return top_50_sen_queries

top_50_doc_queries_bm25_sentences = calculateTop50Documents(queries_scores_bm25_sorted_sentences)


In [71]:
# An example of the first 10 key, value pairs of top-50-sentences for each query data structure
# Here the key is the query_no
# Here the value is a list representing the numbers of the top 50 sentences having
# the highest bm25 scores sorted in descending order

n_items = take(10, top_50_doc_queries_bm25_sentences.items())
print(n_items)


[(1, [143, 372, 93, 708, 159, 690, 0, 533, 108, 67, 97, 51, 626, 114, 295, 99, 1026, 63, 444, 799, 17, 128, 548, 597, 70, 522, 414, 622, 623, 925, 978, 369, 761, 18, 50, 105, 137, 768, 537, 490, 61, 882, 447, 635, 85, 970, 304, 443, 1011, 134]), (2, [27, 23, 295, 7, 26, 336, 163, 493, 327, 46, 269, 0, 19, 400, 89, 242, 92, 229, 232, 414, 135, 506, 34, 190, 250, 187, 168, 315, 170, 174, 127, 185, 476, 64, 305, 17, 299, 192, 450, 188, 121, 109, 223, 317, 354, 49, 504, 434, 455, 277]), (3, [450, 454, 456, 471, 474, 455, 452, 2, 9, 41, 310, 451, 149, 143, 13, 98, 69, 469, 461, 111, 202, 200, 66, 51, 458, 291, 303, 3, 391, 83, 74, 389, 8, 411, 81, 16, 298, 447, 60, 321, 559, 370, 112, 138, 435, 152, 296, 238, 225, 222]), (4, [176, 276, 654, 58, 755, 596, 333, 687, 265, 574, 119, 334, 331, 281, 117, 1, 95, 34, 726, 85, 246, 599, 156, 581, 673, 36, 39, 78, 155, 407, 53, 102, 21, 149, 131, 431, 766, 412, 279, 679, 144, 223, 677, 487, 326, 234, 567, 42, 505, 154]), (5, [345, 85, 0, 94, 683, 892

In [72]:
# Based on the previous result, here is an exmaple of the top sentence scored for query number 1, 
# it's sentence number 143

print("***** Query No. 1 *****\n")
print(queries_text_preprocessed[1])
print("\n")
print("***** Highest scored sentence *****\n")
print(top_50_doc_sentences_queries_bm25[1][143])

***** Query No. 1 *****

['who', 'is', 'the', 'author', 'of', 'the', 'book', 'the', 'iron', 'lady', 'a', 'biography', 'of', 'margaret', 'thatcher']


***** Highest scored sentence *****

['the', 'iron', 'lady', 'a', 'biography', 'of', 'margaret', 'thatcher', 'by', 'hugo', 'young', 'farrar', 'straus', 'giroux', 'the', 'central', 'riddle', 'revealed', 'here', 'is', 'why', 'as', 'a', 'woman', 'in', 'a', 'mans', 'world', 'margaret', 'thatcher', 'evinces', 'such', 'an', 'exclusionary', 'attitude', 'toward', 'women']


### We observe that the highest score sentence has indeed some of the same terms as the query, and it has also the answer to the required query
#### The answer should be "Hugo Young"

In [73]:
def readSentences(documents_text):
    '''
        This function is to preprocess the documents by converting it to sentences.
        It is a helper function for the getSentencesFromTop50DocumentsPlain()
    '''
    documents_text_sentences = []
    
    for i in range(len(documents_text)):
        
        documents_text_sentences_temp = nltk.sent_tokenize(documents_text[i])
        documents_text_sentences.append(documents_text_sentences_temp)
    
    
    documents_text_sentences = [val for sublist in documents_text_sentences for val in sublist]
    
    return documents_text_sentences
    
    

In [74]:
def getSentencesFromTop50DocumentsWithoutTokenization(top_50_doc_queries_bm25, documents_text_not_tokenized):

    top_50_doc_sentences_queries_bm25 = {}

    for key, value in top_50_doc_queries_bm25.items():
        documents_top_50 = []
        for document_id in value:
            documents_top_50.append(documents_text_not_tokenized[document_id])
        documents_top_50_sentences = readSentences(documents_top_50)
        top_50_doc_sentences_queries_bm25[key] = documents_top_50_sentences
            
    return top_50_doc_sentences_queries_bm25

top_50_doc_sentences_queries_bm25_no_tokenization = getSentencesFromTop50DocumentsWithoutTokenization(top_50_doc_queries_bm25, documents_text_not_tokenized)

            
            

In [75]:
# An example of the first key, value pairs of the sentences of the top 50 documents without tokenization for each query data structure
# Here the key is a string representing query no.
# Here the value is a list of strings representing all sentences of the top 50 documents

n_items = take(1, top_50_doc_sentences_queries_bm25_no_tokenization.items())
print(n_items)


[(1, ['There are two aspects of the Margaret Thatcher phenomenon -- neither of which \nis addressed in this splendid British biography -- that every thinking American \nneeds to keep in mind when assessing the achievements of this remarkable woman.', 'First is her contribution to global diplomacy.', 'A quarter of a century after \nDean Acheson\'s remark that "Great Britain has lost an Empire and has not yet \nfound a role" (as Truman\'s secretary of state he would have known), Thatcher \nhas created a noble task for her country: keeping American foreign policy on \ncourse.', 'When the Bush Administration was struggling to acquire its sea legs earlier \nthis year, it was Thatcher who traveled to South Africa to urge reform on P. W. \nBotha and patience from the front-line states.', "It was she who went to Germany \nto try and win Helmut Kohl's cooperation in the modernization of NATO's \nshort-range nuclear arsenal.", 'Above all, Thatcher demonstrated that the \npsychological advantage 

In [76]:
def getRelevantSentences(patterns, top_50_doc_sentences_queries_bm25_no_tokenization):
    '''
        This function is a function to get the relevant documents for each query based on the associated patterns
    '''
    
    relevant_sentences = {}
    
    for i in range(1, len(top_50_doc_sentences_queries_bm25_no_tokenization) + 1):
        
        relevant_sentences[i] = []
        for k in range(len(patterns[str(i)])):
            regex = re.compile(patterns[str(i)][k])
            
            
            for index, sentence in enumerate(top_50_doc_sentences_queries_bm25_no_tokenization[i]):
                
                if regex.search(sentence) != None:
                    
                    relevant_sentences[i].append((index, sentence))
                    
                    
    return relevant_sentences

relevant_sentences_bm25 = getRelevantSentences(patterns, top_50_doc_sentences_queries_bm25_no_tokenization)


In [77]:
# An example of the first key, value pairs of relevant sentences for top 50 documents for each query data structure
# Here the key is a string representing query no.
# Here the value is a list of tuples, each representing the sentence number and the corresponding sentence text 

n_items = take(1, relevant_sentences_bm25.items())
print(n_items)

[(1, [(16, 'In this same revisionist mold, Hugo Young, the \ndistinguished British journalist, has performed a brilliant dissection of the \nnotion of Thatcher as a conservative icon.'), (17, 'In "The Iron Lady," Young traces the winding staircase of fortune that \ntransformed the younger daughter of a provincial English grocer into the \ngreatest woman political leader since Catherine the Great.'), (20, 'To read Young \nis to discover that certain suspect analogies have informed the American \nperception of Thatcher as a conservative heroine.'), (21, "Neither the assumption that Thatcher's road to Damascus parallels Ronald \nReagan's own timely evolution from New Deal enthusiast to conservative \nidealogue, nor the thesis that the Iron Lady took the Tory party by storm in a \npolitical coup a la Barry Goldwater, survives Young's analysis."), (24, "Young makes a persuasive case for the view that Thatcher's less-than-true-blue \nideology was mirrored in her policies."), (28, 'The \nTory

In [78]:
# Here is an exmaple of the top sentences scored for query number 1, 
# The highest is sentence number 268, we then check if it's presented in the relevant sentences

print("***** Query No. 1 *****\n")
print(top_50_doc_queries_bm25_sentences[1])
print("\n")
print("***** Highest scored document *****\n")

for value in relevant_sentences_bm25[1]:
    if value[0] == top_50_doc_queries_bm25_sentences[1][0]:
        print("Sentence number: ", value[0])
        print("Sentence text:", value[1])

***** Query No. 1 *****

[143, 372, 93, 708, 159, 690, 0, 533, 108, 67, 97, 51, 626, 114, 295, 99, 1026, 63, 444, 799, 17, 128, 548, 597, 70, 522, 414, 622, 623, 925, 978, 369, 761, 18, 50, 105, 137, 768, 537, 490, 61, 882, 447, 635, 85, 970, 304, 443, 1011, 134]


***** Highest scored document *****

Sentence number:  143
Sentence text: THE IRON LADY; A Biography of Margaret Thatcher by Hugo Young (Farrar, Straus 
& Giroux) 

The central riddle revealed here is why, as a woman in a man's world, Margaret 
Thatcher evinces such an exclusionary attitude toward women.


### We observe that the highest score sentence for query number 1 is indeed exists in the relevant sentences


### (c)
#### Evaluate the performance of your model using the mean reciprocal rank function (MRR) on the test queries Q:

$$ MRR = \frac{1}{|Q|}\sum_{q=1}^{Q}\frac{1}{rank(q)} $$

#### where rank(q) is the rank of the first relevant sentence for the query q. Use the regex patterns to determine whether a sentence is relevant or not.

In [79]:
def calculateMeanReciprocalRankMRR(top_50_doc_queries_bm25_sentences, relevant_sentences_bm25):
    '''
        This function is a function to calculate the mean rank of all queries
    '''
    
    total_mean_rank = 0
    for i in range(1, len(top_50_doc_queries_bm25_sentences) + 1):
          
        rank = 0
        
        relevant_sentences = []
        for value in relevant_sentences_bm25[i]:
            relevant_sentences.append(value[0])
        
        query_sentences = top_50_doc_queries_bm25_sentences[i]
        for j in range(len(query_sentences)):
            if query_sentences[j] in relevant_sentences:
                rank = j + 1
                break
         
        if rank != 0:
            mean_rank = 1 / rank
            
            total_mean_rank += mean_rank
        
        
    return total_mean_rank / 100

total_mean_rank = calculateMeanReciprocalRankMRR(top_50_doc_queries_bm25_sentences, relevant_sentences_bm25)


In [80]:
# Calculate the mean rank for all the queries

print(total_mean_rank)

0.3929757618757231
